<a href="https://colab.research.google.com/github/Sravani-05/EDA-and-Apache-beam/blob/main/Assignment03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install apache_beam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.9 MB 18.3 MB/s 
     |████████████████████████████████| 270 kB 56.9 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 47 kB 5.7 MB/s 
     |████████████████████████████████| 151 kB 46.7 MB/s 
     |████████████████████████████████| 508 kB 58.6 MB/s 
     |████████████████████████████████| 2.4 MB 62.2 MB/s 
     |████████████████████████████████| 1.0 MB 51.0 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=7c1160144c31009dc15977a4f459e86856d57c2327b3391cdfeb8bf271b1f2b0
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=9529828c2c7b310c505217ffef233d536ffe715fe69824a9b7e87c1f3a650e6d
  Stored in directory: /root/.c

In [2]:
import apache_beam as beam

def SplitRow(element):
  return element.split(',')
 
  
def filter_on_count(element):
  name, count = element
  if count > 30:
    return element
  
def format_output(element):
  name, count = element
  return (name.encode('ascii'),str(count),'Experienced employee')

In [4]:
with beam.Pipeline() as p:
  input_data = (p 
                      | "Read from text file" >> beam.io.ReadFromText('/content/dept_data.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

In [15]:
 accounts_count = (
                      input_data
                      | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                      | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
                      | 'Group and sum1' >> beam.CombinePerKey(sum)
                      | 'count filter accounts' >> beam.Filter(filter_on_count)
                      | 'Regular accounts employee' >> beam.Map(format_output)
                      | 'Write results for account' >> beam.io.WriteToText('data/Account_quick')
                 )

In [6]:
 hr_count = (
                      input_data
                      | 'Get all HR  dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
                      | 'Pair each HR employee with 1' >> beam.Map(lambda record: ("HR, " +record[1], 1))
                      | 'Group and sum1' >> beam.CombinePerKey(sum)
                      | 'count filter accounts' >> beam.Filter(filter_on_count)
                      | 'Regular accounts employee' >> beam.Map(format_output)
                      | 'Write results for HR' >> beam.io.WriteToText('data/HR_quick')
                 )


In [7]:
 finance_count = (
                  input_data
                  | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'Finance')
                  | 'Pair each finance employee with 1' >> beam.Map(lambda record: ("Finance, " +record[1], 1))
                  | 'Group and sum11' >> beam.CombinePerKey(sum)
                  | 'count filter accounts1' >> beam.Filter(filter_on_count)
                  | 'Regular accounts employee1' >> beam.Map(format_output)
                  | 'Write results for finance' >> beam.io.WriteToText('data/Finance_quick')
                   )

In [16]:
!{('head -n 10 data/Account_quick-00000-of-00001')}

head: cannot open 'data/Account_quick-00000-of-00001' for reading: No such file or directory


In [9]:
!{('head -n 10 data/Finance_quick-00000-of-00001')}

head: cannot open 'data/Finance_quick-00000-of-00001' for reading: No such file or directory


In [10]:
class MyTransform(beam.PTransform):
  
  def expand(self, input_coll):
    
    a = ( 
        input_coll
                       | 'Group and sum1' >> beam.CombinePerKey(sum)
                       | 'count filter accounts' >> beam.Filter(filter_on_count)
                       | 'Regular accounts employee' >> beam.Map(format_output)
              
    )
    return a

In [11]:
with beam.Pipeline() as p:
  input_data = (p 
                      | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )
  
  accounts_count = (
                      input_data
                      | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                      | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
                      | 'composite accounts' >> MyTransform()
                      | 'Write results for account' >> beam.io.WriteToText('data/Account')
                 )

  finance_count = (
                  input_data
                  | 'Get all Finance dept persons' >> beam.Filter(lambda record: record[3] == 'Finance')
                  | 'Pair each Finance employee with 1' >> beam.Map(lambda record: ("Finance, " +record[1], 1))
                  | 'composite Finance' >> MyTransform()
                  | 'Write results for Finance' >> beam.io.WriteToText('data/Finance')
            ) 

In [12]:
!{('head -n 10 data/Account-00000-of-00001')}

(b'Accounts, Marco', '31', 'Experienced employee')
(b'Accounts, Rebekah', '31', 'Experienced employee')
(b'Accounts, Itoe', '31', 'Experienced employee')
(b'Accounts, Edouard', '31', 'Experienced employee')
(b'Accounts, Kyle', '62', 'Experienced employee')
(b'Accounts, Kumiko', '31', 'Experienced employee')
(b'Accounts, Gaston', '31', 'Experienced employee')


In [13]:
!{('head -n 10 data/Finance-00000-of-00001')}

(b'Finance, Kumiko', '31', 'Experienced employee')
(b'Finance, Wendy', '31', 'Experienced employee')
(b'Finance, Cristobal', '31', 'Experienced employee')
(b'Finance, Erika', '31', 'Experienced employee')
(b'Finance, Sebastien', '31', 'Experienced employee')
(b'Finance, Valerie', '31', 'Experienced employee')
(b'Finance, Dolly', '31', 'Experienced employee')
(b'Finance, Emily', '31', 'Experienced employee')
(b'Finance, Kaori', '31', 'Experienced employee')
(b'Finance, Hitomi', '31', 'Experienced employee')
